### 

# Part I

In [ ]:
#in order to reduce size of data, we only use random 100000 data of training data as whole data above.
import pandas as pd
import random
from random import randint
 
oldf=open('/kaggle/input/expedia-hotel-recommendations/train.csv','r',encoding='UTF-8')
newf=open('new_meta.csv','w',encoding='UTF-8')
n = 0
# sample(x,y)函数的作用是从序列x中，随机选择y个不重复的元素
resultList = random.sample(range(1,753407),100000)
lines=oldf.readlines()
newf.write(lines[0])
for i in resultList:
    newf.write(lines[i])
    
oldf.close()
newf.close()
meta_data=pd.read_csv('new_meta.csv')
meta_data.head()

In [ ]:
print(len(meta_data))

In [ ]:
meta_data.dtypes

In [ ]:
meta_data.isnull().any()

In [ ]:
#in order to use ML model, we drop the cloumns whose type is object
#we also need to separate x and y 

Y = meta_data['is_booking']
X = meta_data.drop(['date_time','orig_destination_distance','srch_ci','srch_co','is_booking'],axis=1)
X.info()

In [ ]:
#divide train data into 2 parts: "learning" set and "testing" set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=1)
print(len(X_train))
print(len(X_test))

In [ ]:
#first ML algorithm: RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix
clf = RandomForestClassifier(n_estimators=80)
clf.fit(X_train, Y_train)
y_pred=clf.predict(X_test)
acc=accuracy_score(Y_test,y_pred)
#prec=precision_score(Y_test, y_pred,average='micro')
#recall=recall_score(Y_test, y_pred)
#f1_v=f1_score(Y_test, y_pred)
print("Accuracy:" ,acc)
print( "confusion_matrix")
print( confusion_matrix(Y_test, y_pred))
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))
print(clf.feature_importances_)


In [ ]:
#second ML algorithm: GaussianNB()
from sklearn.naive_bayes import GaussianNB
mnb = GaussianNB()
mnb.fit(X_train,Y_train) 
y_predict = mnb.predict(X_test)
acc=accuracy_score(Y_test,y_predict)
#prec=precision_score(Y_test, y_pred,average='micro')
#recall=recall_score(Y_test, y_pred)
#f1_v=f1_score(Y_test, y_pred)
print("Accuracy:" ,acc)
print( "confusion_matrix")
print( confusion_matrix(Y_test, y_predict))
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_predict))


In [ ]:
#Third ML algorithm:Logistic
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, Y_train)
y_predict = lr.predict(X_test)
acc=accuracy_score(Y_test,y_predict)
#prec=precision_score(Y_test, y_pred,average='micro')
#recall=recall_score(Y_test, y_pred)
#f1_v=f1_score(Y_test, y_pred)
print("Accuracy:" ,acc)
print( "confusion_matrix")
print( confusion_matrix(Y_test, y_predict))
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_predict))


In [ ]:
#Forth ML algorithm:KNN
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier()
knn.fit(X_train,Y_train)
y_predict = knn.predict(X_test)
acc=accuracy_score(Y_test,y_predict)
#prec=precision_score(Y_test, y_pred,average='micro')
#recall=recall_score(Y_test, y_pred)
#f1_v=f1_score(Y_test, y_pred)
print("Accuracy:" ,acc)
print( "confusion_matrix")
print( confusion_matrix(Y_test, y_predict))
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_predict))


# Part II

In [ ]:
import pandas as pd
import random
from random import randint
 
oldf=open('/kaggle/input/expedia-hotel-recommendations/test.csv','r',encoding='UTF-8')
newf=open('new_choose.csv','w',encoding='UTF-8')
n = 0
# sample(x,y)函数的作用是从序列x中，随机选择y个不重复的元素
resultList = random.sample(range(1,75342),6000)
lines=oldf.readlines()
newf.write(lines[0])
for i in resultList:
    newf.write(lines[i])
    
oldf.close()
newf.close()
meta_data=pd.read_csv('new_choose.csv')
meta_data.head()

In [ ]:
meta_data.info()

In [ ]:
meta_data.groupby('is_mobile').count()

### is_package(clicking) for each AB-group

In [ ]:
meta_data.groupby('is_mobile')['is_package'].mean()

It appears that there was slight decrease in mobile connection when the booking was generated compare to the control when user connected from others. But while we are certain of the difference in the data, how certain should we be that mobile connection will be worse in the future?

In [ ]:
# Creating an list with bootstrapped means for each AB-group
boot_1d = []
for i in range(1000):
    boot_mean = meta_data.sample(frac = 1,replace = True).groupby('is_mobile')['is_package'].mean()
    boot_1d.append(boot_mean)
    
# Transforming the list to a DataFrame
boot_1d = pd.DataFrame(boot_1d)
    
# A Kernel Density Estimate plot of the bootstrap distributions
boot_1d.plot(kind='density')

In [ ]:
boot_1d.info()

In [ ]:
# Adding a column with the % difference between the two AB-groups
boot_1d['diff'] = (boot_1d[0] - boot_1d[1])/boot_1d[1]*100

# Ploting the bootstrap % difference
ax = boot_1d['diff'].plot(kind='density')
ax.set_title('% difference in is_package between the two AB-groups')

# Calculating the probability that 1-day retention is greater when the gate is at level 30
print('Probability that click/booking is worse when use mobile connection:',(boot_1d['diff'] > 0).mean())